In [0]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt





# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import seaborn as sns
	
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
df_train_data = pd.read_csv("train_data.csv",header=None)
df_train_labels = pd.read_csv("train_labels.csv",header=None)

df_test_data = pd.read_csv("test_data.csv",header=None)




In [0]:
X, X_val, y, y_val = train_test_split(df_train_data, df_train_labels,  test_size=0.1,random_state=0)

In [10]:







columnsD = []
for i in range(1,X.shape[1]+1):
    txt = ""
    if i <= 24:
        txt = "mean_band_"+str(i)        
    elif i <= 24*2:
        txt = "median_band_"+str(i-24)
    elif i <= 24*3:
        txt = "variance_band_"+str(i-24*2)
    elif i <= 24*4:
        txt = "kurtosis_band_"+str(i-24*3)
    elif i <= 24*5:
        txt = "skewness_band_"+str(i-24*4)
    elif i <= 24*6:
        txt = "min_band_"+str(i-24*5)
    elif i <= 24*7:
        txt = "max_band_"+str(i-24*6)
    elif i <= (24*7)+12:
        txt = "mean_pc_"+str(i-((24*7)))
    elif i <= (24*7)+(12*2):
        txt = "stan_deriv_pc_"+str(i-((24*7)+12))
    elif i <= (24*7)+(12*3):
        txt = "min_pc_"+str(i-((24*7)+(12*2)))
    elif i <= (24*7)+(12*4):
        txt = "max_pc_"+str(i-((24*7)+(12*3)))
    elif i <= (24*7)+(12*4)+12:
        txt = "mean_coef_"+str(i-((24*7)+(12*4)))
    elif i <= (24*7)+(12*4)+(12*2):
        txt = "stan_deriv_coef_"+str(i-((24*7)+(12*4)+12))
    elif i <= (24*7)+(12*4)+(12*3):
        txt = "min_coef_"+str(i-((24*7)+(12*4)+(12*2)))
    elif i <= (24*7)+(12*4)+(12*4):
        txt = "max_coef_"+str(i-((24*7)+(12*4)+(12*3)))
    
        
    columnsD.append(txt)
    
df_test_data.columns = columnsD
X.columns = columnsD

#In the future it can be expanded to more features
columnsL = []
for i in range(1,y.shape[1]+1):
    columnsL.append("Column_"+str(i))
y.columns = columnsL


y["Column_1"] = y["Column_1"].astype('category')


union = pd.concat([X, y ], axis = 1)
info = union.Column_1.value_counts()
result = pd.DataFrame()
print(info)
info[10]
for cat in range(1,11):
  
    data_to_res = union[union.Column_1 == cat]
          #print(data_to_res)
    samples = info[cat]
    if(info[cat] < 500):
        samples = 500

    df = resample(data_to_res, replace=True, n_samples=samples)

    result = pd.concat([result,df])


print(result.Column_1.value_counts())
result = result.sample(n=result.shape[0])
#print(result["Column_1"])
y = pd.DataFrame(result["Column_1"])

y.columns = columnsL


y["Column_1"] = y["Column_1"].astype('category')
X = pd.DataFrame(result[columnsD])


1     1966
2      561
3      299
4      233
6      232
5      182
8      165
7      129
9       85
10      74
Name: Column_1, dtype: int64
1     1966
2      561
10     500
9      500
8      500
7      500
6      500
5      500
4      500
3      500
Name: Column_1, dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Creating Training and Test Sets 

In [0]:
X = X.values
y = y.values.reshape(len(y))
X_test_unlabeled = df_test_data.values








# Scaling data

In [0]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaler.fit(X)
X= scaler.transform(X)
X_test_unlabeled = scaler.transform(X_test_unlabeled)
X_val= scaler.transform(X_val)


# Grid Search of different Models

In [0]:


from sklearn.model_selection import GridSearchCV

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=6, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.metrics import log_loss




from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier



models1 = {
    'XGBClassifier': XGBClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'LogisticRegression' : LogisticRegression(),
    'SVC': SVC(class_weight='balanced',probability=True)
}

params1 = {
    'XGBClassifier': { 'n_estimators': [512], 'learning_rate': [0.2,0.5,0.8] },
    'RandomForestClassifier': { 'n_estimators': [512] },
    'AdaBoostClassifier':  { 'n_estimators': [512] },
    'GradientBoostingClassifier': { 'n_estimators': [256,512], 'learning_rate': [0.2,0.8, 1.0] },
    'LogisticRegression': {'penalty': ['l2'], 'C' : [0.15,1,2,5]},
    'SVC': {'kernel': ['linear'], 'C': [1, 5, 8]}
}

In [0]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(X, y, scoring='accuracy', n_jobs=8)

Running GridSearchCV for XGBClassifier.
Fitting 3 folds for each of 3 candidates, totalling 9 fits


In [10]:
helper1.score_summary(sort_by='max_score')

XGBClassifier
RandomForestClassifier
AdaBoostClassifier
GradientBoostingClassifier
LogisticRegression
SVC


,estimator,min_score,mean_score,max_score,std_score,C,kernel,learning_rate,n_estimators,penalty
0,XGBClassifier,0.892562,0.898278,0.90553,0.00540445,NaN,NaN,0.2,512,NaN
1,XGBClassifier,0.889807,0.893526,0.899078,0.00400069,NaN,NaN,0.5,512,NaN
2,XGBClassifier,0.889399,0.892761,0.898618,0.00415638,NaN,NaN,0.8,512,NaN
6,GradientBoostingClassifier,0.894858,0.895358,0.895853,0.000406179,NaN,NaN,0.2,512,NaN
5,GradientBoostingClassifier,0.890725,0.893827,0.895824,0.00222301,NaN,NaN,0.2,256,NaN
3,RandomForestClassifier,0.885216,0.889083,0.89447,0.00392792,NaN,NaN,NaN,512,NaN
8,GradientBoostingClassifier,0.852995,0.862711,0.870124,0.00717976,NaN,NaN,0.8,512,NaN
7,GradientBoostingClassifier,0.853456,0.862253,0.868747,0.00645122,NaN,NaN,0.8,256,NaN
10,GradientBoostingClassifier,0.783387,0.802195,0.832874,0.0218778,NaN,NaN,1,512,NaN
9,GradientBoostingClassifier,0.782928,0.801279,0.828742,0.0197824,NaN,NaN,1,256,NaN


In [0]:
clf1 = GradientBoostingClassifier(learning_rate=0.2, n_estimators=512)
clf2 = XGBClassifier(learning_rate = 0.2, n_estimators = 512)
clf3 = SVC(C=5, kernel='linear',probability=True, class_weight= 'balanced')
lr = LogisticRegression()
#sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
#                          use_probas=True,
#                          average_probas=False,
#                          meta_classifier=lr)

from sklearn.ensemble import VotingClassifier
sclf = VotingClassifier(estimators=[
    ('lr', clf1), ('xgb', clf2), ('svc', clf3)],
                         voting='soft')

name_best_model = 'stacked_LR_XGB_SVC'

best_model = sclf





best_model= best_model.fit(X,y)





# Validation (Give how good is our model with unseen data)

In [14]:




sclf.score(X_val,y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6704805491990846

# Taking the full X and y now to train the model to fit thes semisupervised and also the final model.

In [0]:
X = df_train_data.values
y = df_train_labels.values.reshape(len(df_train_labels))
X_test_unlabeled = df_test_data.values

In [0]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X= scaler.transform(X)
X_test_unlabeled = scaler.transform(X_test_unlabeled)

## Semi-supervised learning (Pseudo Labeling)

In [17]:
y_test_unlabeled = best_model.predict(X_test_unlabeled)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Concatenate training labels with test set pseudo labels

In [0]:
y_semisupervised = np.concatenate((y, y_test_unlabeled))

### Concatenate training features with test features

In [0]:
X_semisupervised = np.concatenate((X, X_test_unlabeled))

### Train the model including pseudo labels

In [20]:
best_model_ss= best_model.fit(X_semisupervised,y_semisupervised)



print('Cross Validation Accuracy of {} (best classifier) with SEMI-SUPERVISED LEARNING: {:.3f}'
       .format(name_best_model,accuracy_best_model))

from sklearn.metrics import classification_report
print(classification_report(y, best_model.predict(X)))

KeyboardInterrupt: ignored